[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/02-dynamic-routes.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/02-dynamic-routes.ipynb)

# Dynamic Routes

In semantic-router there are two types of routes that can be chosen. Both routes belong to the `Route` object, the only difference between them is that _static_ routes return a `Route.name` when chosen, whereas _dynamic_ routes use an LLM call to produce parameter input values.

For example, a _static_ route will tell us if a query is talking about mathematics by returning the route name (which could be `"math"` for example). A _dynamic_ route can generate additional values, so it may decide a query is talking about maths, but it can also generate Python code that we can later execute to answer the user's query, this output may look like `"math", "import math; output = math.sqrt(64)`.

***⚠️ Note: We have a fully local version of dynamic routes available at [docs/05-local-execution.ipynb](https://github.com/aurelio-labs/semantic-router/blob/main/docs/05-local-execution.ipynb). The local 05 version tends to outperform the OpenAI version we demo in this notebook, so we'd recommend trying [05](https://github.com/aurelio-labs/semantic-router/blob/main/docs/05-local-execution.ipynb)!***

## Installing the Library

In [12]:
!pip install -qU "semantic-router[local]==0.0.20"
!pip install unifyai
!pip install openai



  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      *** scikit-build-core 0.9.2 using CMake 3.29.2 (wheel)
      *** Configuring CMake...
      2024-04-29 12:24:08,789 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
      loading initial cache file C:\Users\indir\AppData\Local\Temp\tmppik1ey4m\build\CMakeInit.txt
      -- Building for: NMake Makefiles
      CMake Error at CMakeLists.txt:3 (project):
        Running
      
         'nmake' '-?'
      
        failed with:
      
         no such file or directory
      
      
      CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
      CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
      -- Configuring incomplete, errors occurred!
      
      *** CMake configuration failed
      [end of output]
  
  note: This error or


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\indir\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\indir\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Initializing Routes and RouteLayer

Dynamic routes are treated in the same way as static routes, let's begin by initializing a `RouteLayer` consisting of static routes.

In [10]:
from semantic_router import Route

# Define routes for Math and Coding
math_route = Route(
    name="math",
    utterances=[
        "solve for x in the equation",
        "what is the integral of",
        "how to calculate the derivative",
        "mathematical proofs",
        "how do you find the percentage of this number"
    ],
)

coding_route = Route(
    name="coding",
    utterances=[
        "how to write a for loop in Python",
        "explain the use of classes in Java",
        "what is recursion in programming",
        "how do i optimise this problem using hash tables",
        "suggest a more efficient data structure for this problem"
    ],
)

# List of all routes
routes = [math_route, coding_route]

We initialize our `RouteLayer` with our `encoder` and `routes`. We can use popular encoder APIs like `CohereEncoder` and `OpenAIEncoder`, or local alternatives like `FastEmbedEncoder`.

In [11]:
import os
from getpass import getpass
from semantic_router import RouteLayer
from semantic_router.encoders import HuggingFaceEncoder

encoder = HuggingFaceEncoder()

rl = RouteLayer(encoder=encoder, routes=routes)

ImportError: Please install fastembed to use FastEmbedEncoder. You can install it with: `pip install 'semantic-router[fastembed]'`

We run the solely static routes layer:

In [4]:
rl("Solve the equation 5-x=12 for x?")

RouteChoice(name='math', function_call=None, similarity_score=None)

In [13]:
from unify import Unify
# Environment setup for API keys
os.environ["UNIFY_KEY"] =  getpass("Enter Unify API Key: ")

In [ ]:
import os
from unify import ChatBot

def test_unify_endpoint():
    try:
        # Set up the ChatBot with a known working endpoint and API key
        agent = ChatBot(
            api_key=os.environ["UNIFY_KEY"],
            endpoint="gpt-4@anyscale"  # Adjust this if necessary
        )
        
        # Example prompt to test the service
        test_prompt = "What is 2+2?"
        
        # Making a request to the Unify service
        response = agent.run(test_prompt)
        print("Response from Unify:", response)
        
    except Exception as e:
        print("Failed to connect to Unify endpoint:", str(e))

if __name__ == "__main__":
    test_unify_endpoint()


In [44]:
#Unify process query
def process_query(query):
    route_choice = rl(query)
    print(f"Route chosen: {route_choice.name}")

    if route_choice.name == "math":
        # Initialize Unify with the endpoint for math queries
        unify = Unify(
            api_key=os.environ["UNIFY_KEY"],
            endpoint="llama-2-13b-chat@anyscale"  # Use the correct endpoint for math queries, other models not working
        )
        # Generate the response using Unify
        response = unify.generate(user_prompt=query)
        return response

    elif route_choice.name == "coding":
        # Initialize Unify with the endpoint for coding queries
        unify = Unify(
            api_key=os.environ["UNIFY_KEY"],
            endpoint="codellama-34b-instruct@anyscale"  # Use the correct endpoint for coding queries
        )
        # Generate the response using Unify
        response = unify.generate(user_prompt=query)
        return response

    else:
        return "This query does not fall under a supported category."



In [45]:
# Process query test
print(process_query("Solve the equation 5-x=12 for x?"))

Route chosen: math
Sure! To solve the equation 5 - x = 12, we can add x to both sides of the equation.

5 - x = 12

Adding x to both sides gives us:

5 = 12 + x

Now we can subtract 12 from both sides:

5 - 12 = x

This simplifies to:

-7 = x

So the solution to the equation 5 - x = 12 is x = -7.


In [39]:
print(process_query("Write a for loop in python that appends numbers from 1-10 in a list"))

Route chosen: coding

Here is the code:

```py
my_list = []
for num in range(1, 11):
    my_list.append(num)
```

This will create a list containing the numbers 1 to 10 inclusively with each number in its own cell.

_Note: The range() function is used to generate a sequence of numbers. The range() function when passed only one argument (in this case, 11) will return the sequence of numbers starting from 0 and going up to, but not including, the number passed, which is 11 in this case. By passing 1 as the starting number (range(1, 11)), we get the numbers from 1 to 10._

Now you have a list with numbers from 1 to 10 in it. Here are some ways to use it:

```py
print(my_list[0])  # prints the first element which is 1
print(my_list[9])  # prints the last element which is 10
print(my_list)  # prints the whole list [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
```

Let me know if you have further questions! 😊
